In [1]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere
from random import getrandbits
import pickle

RETAINED_CHAR = '_'

In [2]:

class CryptoCircuit(QuantumCircuit):
    def __init__(self, size=7):
        # Each qubit representing a bit in ASCII (7-bit)
        super().__init__(size)

    def initialize(self, code:str):
        code = code[::-1] # reverse order
        for i in range(len(code)):
            self.x(i) if code[i]!='0' else self.i(i)

    def add_gate(self, gate:bool):
        self.h(self.qubits) if gate else self.i(self.qubits)

    def get_measurements(self):
        self.measure_all()
        sim = Aer.get_backend("qasm_simulator")
        counts = execute(self, sim, shots=1).result().get_counts()
        return list(counts)[0]

    def visualize(self):
        sim = Aer.get_backend("statevector_simulator")
        statevector = execute(self, sim, shots=1).result().get_statevector()
        display(plot_bloch_multivector(statevector))
        # display(plot_state_qsphere(statevector))
    
    def draw(self, output=""):
        return ("You are not allowed to print the circuit..")



In [3]:

class ProtocolBB84:

    def receiver(circuits:list[CryptoCircuit]) -> tuple[list[str], list[int]]:
        letters, gates = [], []
        for circuit in circuits:
            # Skip the already found letters
            if circuit == None:
                letters.append(RETAINED_CHAR)
                gates.append(None)
                continue

            # Choose a random receiving basis
            gates.append(getrandbits(1))
            circuit.add_gate(gates[-1])

            # Measure the qubits to get the received binary code
            letter = circuit.get_measurements()
            # Get letter from binary code
            letters.append(chr(int(letter,2)))

        return letters, gates


    def classicalChannel(sender_gates, receiver_gates):
        correct_chars = []
        for i, (sent,received) in enumerate(zip(sender_gates, receiver_gates)):
            if sent == received :
                correct_chars.append(i)
        return correct_chars



In [5]:

# Receive - read from file
with open("quantum_channel.txt", "br") as f:
    circuits_in = f.read().split(b'\n\n')[:-1]

with open("classical_channel.txt", "r") as f:
    sender_gates = [int(gate) for gate in f.read()]


# In real example this loop would not be possible since the quantum state would colapse
final_message = RETAINED_CHAR *len(sender_gates)
while '_' in final_message:
    received_message, receiver_gates = ProtocolBB84.receiver([pickle.loads(circuit) for circuit in circuits_in])

    correct_chars = ProtocolBB84.classicalChannel(sender_gates, receiver_gates)
    received_correctly = ''.join([letter if i in correct_chars else RETAINED_CHAR for i,letter in enumerate(received_message)])
    
    final_message = ''.join([ letter if letter!=RETAINED_CHAR else received_correctly[i] for i, letter in enumerate(final_message) ])

    print(received_correctly, '->', final_message)



B_8_ i____Q_an__m _e_ _is_ri___i_n__r__oc_l _e_y_n_ o_ ____l___ng th_o_em. -> B_8_ i____Q_an__m _e_ _is_ri___i_n__r__oc_l _e_y_n_ o_ ____l___ng th_o_em.
BB84 is a__ua__u__K____i__ri___io__p_o_o____re_yi_g _n____clon_n_ __e_r___ -> BB84 is a_Quan_um Ke_ _is_ri___ion_pro_oc_l re_ying on ___clon_ng theorem.
B__4 _s a___an_um Ke_ _i_tr__ut___ __oto_o_ rely___ o__no-__o__ng ______m_ -> BB84 is a_Quan_um Ke_ _istri_ution protocol relying on no-clon_ng theorem.
BB_4 is__ Q_an_um _e_ _i_trib___o_____toc_l __ly_____n no-c__n_n_ ___o__m. -> BB84 is a Quan_um Ke_ _istribution protocol relying on no-clon_ng theorem.
BB_________ua_t_m______i_tri______ prot___l re_ying o__n__c_o_i___th_orem_ -> BB84 is a Quantum Ke_ _istribution protocol relying on no-cloning theorem.
___4 __ ____an__m_Ke__D__t___ut__n _r_t___l r__y__g __ _o_cl_____ t_e_rem. -> BB84 is a Quantum Ke_ Distribution protocol relying on no-cloning theorem.
_B84_is__ Q__ntum_Key __str_b____n_pr___c__ __ly__g____n_-clonin_ _he__e_. -> BB84